In [1]:
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset, Subset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch

In [2]:
import torch_directml
dml = torch_directml.device()
print(dml)

privateuseone:0


In [3]:
# Load the dataset with data augmentation and normalizing
class Cifar10(Dataset):
    def __init__(self):
        super().__init__()
        transform_train = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.RandomAutocontrast(),
            transforms.ToTensor(),
            transforms.Resize((128, 128)), # B2 for resizing image
            transforms.Normalize(mean=[0.4854, 0.4567, 0.4062], std=[0.2291, 0.2249, 0.2253])
        ])
        self.data = ImageFolder("clean_train", transform=transform_train)

    def __len__(self):
        return self.data.__len__()

    def __getitem__(self, idx):
        features, label = self.data[idx]
        return features, label

    def split_data(self):
        indices = np.arange(len(self.data))
        targets = np.array(self.data.targets)
        train_indices, val_indices = train_test_split(indices,
                                                       test_size=0.2,
                                                       random_state=42,
                                                       stratify=targets)
        final_train_set = Subset(self.data, train_indices)
        final_val_set = Subset(self.data, val_indices)
        return final_train_set, final_val_set
        
train_data = Cifar10()

In [4]:
# Split the dataset
train_set, val_set = train_data.split_data()

In [5]:
# Build the CNN
class CnnCifar(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), # B4. Padding in each CONV layer
            nn.BatchNorm2d(64),
            nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), # B4. Padding in each CONV layer
            nn.BatchNorm2d(128),
            nn.LeakyReLU(negative_slope=0.01),
            nn.MaxPool2d(kernel_size=2),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.2)
        )
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x


In [6]:
# Feed training data into the model
batch_size = 128
train_load = DataLoader(train_set, shuffle=True,  batch_size=batch_size,
                        num_workers=4, pin_memory=True, persistent_workers=True)
val_load   = DataLoader(val_set,   shuffle=False, batch_size=batch_size,
                        num_workers=4, pin_memory=True, persistent_workers=True)
cnn = CnnCifar().to(dml)
criterion = nn.CrossEntropyLoss().to(dml)
optimizer = optim.Adam(cnn.parameters(), lr=0.001)

for epoch in range(30):
    run_amount = 0
    for images, labels in train_load:
        images = images.to(memory_format=torch.channels_last).to(dml, non_blocking=True)
        labels = labels.to(dml, non_blocking=True)
        
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        run_amount += loss.item()
    avg_loss = run_amount / len(train_load)
    print(f"Epoch [{epoch+1}/30], Loss: {avg_loss:.4}")

C:\Users\ancho\anaconda3\envs\tf_env\Lib\site-packages\torch\optim\adam.py:534: UserWarning: The operator 'aten::lerp.Scalar_out' is not currently supported on the DML backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at C:\__w\1\s\pytorch-directml-plugin\torch_directml\csrc\dml\dml_cpu_fallback.cpp:17.)
  torch._foreach_lerp_(device_exp_avgs, device_grads, 1 - beta1)


Epoch [1/30], Loss: 1.952
Epoch [2/30], Loss: 1.804
Epoch [3/30], Loss: 1.756
Epoch [4/30], Loss: 1.723
Epoch [5/30], Loss: 1.698
Epoch [6/30], Loss: 1.674
Epoch [7/30], Loss: 1.658
Epoch [8/30], Loss: 1.639
Epoch [9/30], Loss: 1.614
Epoch [10/30], Loss: 1.609
Epoch [11/30], Loss: 1.592
Epoch [12/30], Loss: 1.572
Epoch [13/30], Loss: 1.557
Epoch [14/30], Loss: 1.546
Epoch [15/30], Loss: 1.535
Epoch [16/30], Loss: 1.528
Epoch [17/30], Loss: 1.519
Epoch [18/30], Loss: 1.506
Epoch [19/30], Loss: 1.502
Epoch [20/30], Loss: 1.497
Epoch [21/30], Loss: 1.488
Epoch [22/30], Loss: 1.48
Epoch [23/30], Loss: 1.474
Epoch [24/30], Loss: 1.465
Epoch [25/30], Loss: 1.466
Epoch [26/30], Loss: 1.459
Epoch [27/30], Loss: 1.446
Epoch [28/30], Loss: 1.438
Epoch [29/30], Loss: 1.442
Epoch [30/30], Loss: 1.431


In [7]:
# Evaluate
from torchmetrics import Recall, Precision

cnn.eval()
metric_recall = Recall(
    task = "multiclass", num_classes=10, average=None
).to(dml)

metric_precision = Precision(
    task = "multiclass", num_classes=10, average=None
).to(dml)

with torch.no_grad():
    for images, labels in val_load:
        images = images.to(memory_format=torch.channels_last).to(dml, non_blocking=True)
        labels = labels.to(dml, non_blocking=True)

        outputs = cnn(images)
        _, preds = torch.max(outputs, 1)

        metric_recall(preds, labels)
        metric_precision(preds, labels)

recall = metric_recall.compute()
precision = metric_precision.compute()

recall_per_class = {
    k: recall[v].item()
    for k, v
    in train_data.data.class_to_idx.items()
}

precision_per_class = {
    k: precision[v].item()
    for k, v
    in train_data.data.class_to_idx.items()
}

print(recall_per_class)
print(precision_per_class)

{'airplane': 0.5160000324249268, 'automobile': 0.6260000467300415, 'bird': 0.3720000088214874, 'cat': 0.2670000195503235, 'deer': 0.43300002813339233, 'dog': 0.5960000157356262, 'frog': 0.5630000233650208, 'horse': 0.5220000147819519, 'ship': 0.6970000267028809, 'truck': 0.6800000071525574}
{'airplane': 0.5746102333068848, 'automobile': 0.5711678862571716, 'bird': 0.4320557415485382, 'cat': 0.392070472240448, 'deer': 0.48004433512687683, 'dog': 0.407382071018219, 'frog': 0.7171974778175354, 'horse': 0.5965714454650879, 'ship': 0.5847315192222595, 'truck': 0.5453087091445923}
